# Python/FORTRAN interfacing teaser

This is a collection of general ideas and starting points on how to simplify the further development of ocean, atmosphere and climate models. This notebook is meant to provide examples on how to access compiled FORTRAN routines using Python on-board packages/features. Shared library compiling, shared library function calling and multi-dimensional array manipulation are covered here. This document might serve as discussion basis for the development and application of best practices in ocean/climate model development.

## a subjective ocean modeler's perspective

ocean model code usually consists of a main program unit

```fortran
!--------------------------- main.f90 -----------------------------------
 PROGRAM main

  ! declaration

  ! initialization

  ! main loop
  do it = 1,itend
    CALL update_external_forcing
    CALL numerical_integration
    CALL write_disk_output
  end do

  ! restart preparation
    
 END PROGRAM main
!------------------------------------------------------------------------
```

and many subroutines organized in 

```fortran
!------------------------------ subroutines.f90 -------------------------

 ...

 SUBROUTINE square_root(a)
  IMPLICIT NONE
  real :: a
  a = SQRT(a)
 END SUBROUTINE square_root

 ...
 
!------------------------------------------------------------------------
```

### testing changes to an ocean model code usually involves

* changing of the FORTRAN code
* compiling of complete model code
* setup of a test experiment directory structure
* job submission into some HPC queue
* ... waiting time ...
* drawing conclusions based on output
* probably start over ...

would be very useful if it could be shortened to

* change FORTRAN code
* compile only subroutine
* execute and draw conclusions
* start over ...

## Shared library compiling

ask your compiler's documentation about how to do it!

In [1]:
%%writefile subroutines.f90

SUBROUTINE square(a)
    IMPLICIT NONE
    real :: a
    a = a**2
END SUBROUTINE square

SUBROUTINE square_root(b)
    IMPLICIT NONE
    real :: b
    b = SQRT(b)
END SUBROUTINE square_root


Overwriting subroutines.f90


In [2]:
%%script bash
# echo `which gfortran` && gfortran --version
gfortran -shared -fpic subroutines.f90 -o subroutines.so && ls -lrth

total 40K
-rw-r--r-- 1 jovyan jovyan  156 Dec  5 07:06 README.md
drwxr-xr-x 1 jovyan jovyan 4.0K Dec  5 07:06 binder
-rw-r--r-- 1 jovyan root    17K Dec  5 07:34 lunch-seminar-teaser.ipynb
-rw-r--r-- 1 jovyan root    191 Dec  5 07:34 subroutines.f90
-rwxr-xr-x 1 jovyan root   7.4K Dec  5 07:34 subroutines.so


unix tools are useful...

In [3]:
!readelf --symbols subroutines.so


Symbol table '.dynsym' contains 12 entries:
   Num:    Value          Size Type    Bind   Vis      Ndx Name
     0: 0000000000000000     0 NOTYPE  LOCAL  DEFAULT  UND 
     1: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND __cxa_finalize
     2: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND _ITM_registerTMCloneTable
     3: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND _ITM_deregisterTMCloneTab
     4: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND __gmon_start__
     5: 0000000000201020     0 NOTYPE  GLOBAL DEFAULT   18 _edata
     6: 0000000000201028     0 NOTYPE  GLOBAL DEFAULT   19 _end
     7: 00000000000005b9    31 FUNC    GLOBAL DEFAULT    9 square_root_
     8: 0000000000000488     0 FUNC    GLOBAL DEFAULT    6 _init
     9: 0000000000201020     0 NOTYPE  GLOBAL DEFAULT   19 __bss_start
    10: 00000000000005d8     0 FUNC    GLOBAL DEFAULT   10 _fini
    11: 000000000000059a    31 FUNC    GLOBAL DEFAULT    9 square_

Symbol table '.symtab' contains 49 entries

## Python/FORTRAN interfacing
Don't worry, numpy will handle most of this for you! However,
* https://docs.python.org/3/library/ctypes.html

In [4]:
import ctypes as ctypes
fortran_routines = ctypes.CDLL('./subroutines.so')

In [5]:
python_float = 4.0

In [6]:
ctypes_float = ctypes.c_float(python_float)
ctypes_pointer = ctypes.pointer(ctypes_float)

_ = fortran_routines.square_(ctypes_pointer)
print(ctypes_pointer.contents,':',python_float**2)

c_float(16.0) : 16.0


What happens here?

In [7]:
_ = fortran_routines.square_root_(ctypes_pointer)
print(ctypes_pointer.contents,':',python_float**0.5)

c_float(4.0) : 2.0


## Multi-dimensional array example

* https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html
* https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.ctypes.html

In [8]:
%%writefile subroutines.f90

SUBROUTINE square(a,ij,ik)
    IMPLICIT NONE
    integer :: ij,ik
    real, dimension(ij,ik) :: a
    a = a**2
END SUBROUTINE square


Overwriting subroutines.f90


In [9]:
%%script bash
gfortran -shared -fpic subroutines.f90 -o subroutines.so && ls -lrth
#readelf --symbols subroutines.so

total 40K
-rw-r--r-- 1 jovyan jovyan  156 Dec  5 07:06 README.md
drwxr-xr-x 1 jovyan jovyan 4.0K Dec  5 07:06 binder
-rw-r--r-- 1 jovyan root    17K Dec  5 07:34 lunch-seminar-teaser.ipynb
-rw-r--r-- 1 jovyan root    134 Dec  5 07:34 subroutines.f90
-rwxr-xr-x 1 jovyan root   7.3K Dec  5 07:34 subroutines.so


In [10]:
import numpy as np
fortran_routines = np.ctypeslib.load_library('subroutines.so','./').square_

In [11]:
target_array = np.random.random((2,2))
print(target_array)

[[0.33141847 0.84538779]
 [0.11465003 0.90672487]]


In [12]:
target_original = target_array.copy()

In [13]:
target_array = np.asfortranarray(target_array,dtype=np.float32)
print(target_array.flags)

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False



In [14]:
ik = np.array(target_array.shape[0],dtype=np.int32)
ij = np.array(target_array.shape[1],dtype=np.int32)

In [15]:
fortran_routines(target_array.ctypes,ik.ctypes,ij.ctypes)

3

In [16]:
print(target_array)

[[0.10983821 0.71468043]
 [0.01314463 0.82215   ]]


In [17]:
print(target_original**2)

[[0.1098382  0.71468051]
 [0.01314463 0.82214999]]


# Caveats: Beware of datatypes!
* https://docs.scipy.org/doc/numpy/user/basics.types.html
* e.g. https://northstar-www.dartmouth.edu/doc/solaris-forte/manuals/fortran/prog_guide/11_cfort.html

and use FORTRAN wrappers with ISO_C_BINDING declarations...

* get your datatypes clear!
* name mangling
* pass-by-value functionality

In [18]:
%%writefile subroutines.f90

SUBROUTINE sq1(a,ij,ik)
    use iso_c_binding, only: c_int, c_double
    IMPLICIT NONE
    integer(c_int) :: ij,ik
    real(c_double), dimension(ij,ik) :: a
    a(:,:) = a**2
END SUBROUTINE sq1


Overwriting subroutines.f90


# Questions

* possible to create thin interface to existing modules with ctypes only-use?